In [ ]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt
from scipy import integrate

import csv
import time
import struct

In [ ]:
def readApprox(filename):
    fp = open(filename,"r")
    csvFile = csv.reader(fp,delimiter=",")
    
    firstRow = next(csvFile)
    x = numpy.array(firstRow,dtype=numpy.float64)
    
    numRows = 0
    allData = []
    approx = numpy.arange(0)
    for line in csvFile:
        line = numpy.array(line,dtype=numpy.float64)
        allData.append(line)
        numRows += 1

    approx = numpy.zeros((numRows,len(allData[0])-1))
    timeSteps = numpy.zeros(numRows)
    numRows = 0
    for line in allData:
        approx[numRows,:] = line[1:]
        timeSteps[numRows] = line[0]
        numRows += 1
            
    fp.close()
    
    return(x,timeSteps,approx)

def readBinary(filename):
    fp = open("../build-butterflyVWaspPDE-Desktop-Debug/approximation.bin","rb")

    # Read in the polynomial degree
    N = fp.read(4)
    N = int.from_bytes(N,byteorder='little')

    # Read in the size of the state space
    S = fp.read(4)
    S = int.from_bytes(S,byteorder='little')

    # Read the abscissa
    x = fp.read(8*(N+1))
    x = numpy.array(struct.unpack('{0}d'.format(N+1),x))

    allData = []
    allTime = []
    while(True):
        t = fp.read(8)
        if(len(t)>0):
            allTime.append(struct.unpack('d',t)[0])
            state = numpy.array(struct.unpack('{0}d'.format(N+2),fp.read(8*(N+2))))
            allData.append(state)
        else:
            break


    fp.close()
    
    
    timeSteps = numpy.array(allTime)
    approx    = numpy.zeros((len(allData),N+1))
    wasps     = numpy.zeros(len(allData))
    row = 0
    for line in allData:
        approx[row,0:(N+1)] = line[0:(N+1)]
        wasps[row] = line[N+1]
        row += 1

    return([x,timeSteps,approx,wasps])


In [ ]:
#[x,timeSteps,approx] = readApprox("../build-butterflyVWaspPDE-Desktop-Debug/approximation.csv")
#print("Final Time: {0:.3f}".format(timeSteps[-1]))

[x,timeSteps,approx,wasps] = readBinary("../build-butterflyVWaspPDE-Desktop-Debug/approximation.csv")
x = (x+1.0)*0.5
print("Final Time: {0:.3f}".format(timeSteps[-1]))



In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 1
%matplotlib notebook

In [ ]:
%autoreload 1
%matplotlib notebook


t    = 0
row  = 0
skip = 10;

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
plt.ion()
#fig.show()
for timeIndex in range(0,approx.shape[0],skip):
    #print(butterFly)
    
    axes.clear()
    axes.set_xlabel("x")
    axes.set_ylabel("Butterflies")
    axes.set_title("Butterflies at t={0:.5f}".format(timeSteps[timeIndex]))
    #axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
    axes.set_xlim(-0.1, 1.1)
    axes.set_ylim(-0.1, 1.1)

    butterflyPlot = axes.plot(x,approx[timeIndex],'r')
    fig.canvas.draw()
    time.sleep(0.0)





In [ ]:
%matplotlib inline
%autoreload 0
fig = plt.figure()
fig.subplots_adjust(bottom=-1.0,top=1.0)

axes = fig.add_subplot(2,1,1)

[XGrid,TGrid] = numpy.meshgrid(x,timeSteps)
cp = axes.contourf(XGrid,TGrid,approx,cmap='gist_yarg') # Greens BuGn
fig.colorbar(cp) # Add a colorbar to a plot
axes.set_title('Butterfly Density')
axes.set_xlabel(r'$\theta$')
axes.set_ylabel('Time')

axes1 = fig.add_subplot(2, 1, 2)
axes1.plot(timeSteps,wasps)
axes1.set_title('Density of Wasp Population')
axes1.set_xlabel('Time')
axes1.set_ylabel('Wasp Density')

plt.show()
